<a href="https://colab.research.google.com/github/dharma610/Project/blob/master/SIR_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.integrate import odeint
import numpy as np
import pandas as pd
from random import sample
import matplotlib.pyplot as plt
%matplotlib inline 
!pip install mpld3
import mpld3
mpld3.enable_notebook()

     |████████████████████████████████| 1.0MB 2.9MB/s 
  Created wheel for mpld3: filename=mpld3-0.5.1-cp36-none-any.whl size=364065 sha256=ebb81a02167c126f5e47756bf385a7575621baa2f3c1aefb8d6f798689ec67d2
  Stored in directory: /root/.cache/pip/wheels/38/68/06/d119af6c3f9a2d1e123c1f72d276576b457131b3a7bf94e402
Successfully built mpld3


In [ ]:
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [ ]:
def plotsir(t, S, I, R):
  f, ax = plt.subplots(1,1,figsize=(10,4))
  ax.plot(t, S, 'b', alpha=0.7, linewidth=2, label='Susceptible')
  ax.plot(t, I, 'y', alpha=0.7, linewidth=2, label='Infected')
  ax.plot(t, R, 'g', alpha=0.7, linewidth=2, label='Recovered')
  ax.set_xlabel('Time (days)')
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend()
  legend.get_frame().set_alpha(0.5)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  plt.show();

In [ ]:
""" gamma is a list of 1000 random number in (0,1) """

import random
gamma = []
for i in range(1000):
    gamma.append(random.uniform(0, 1))
    


In [ ]:
""" beta is twice of each of 1000 gamma values """
beta = []
for i in range(1000):
  beta.append(2*gamma[i])  

In [ ]:
""" P, Q, M are the list of S, I, R respectievely calculated for 100 days for each of  1000 sets of parameter (beta,gamma)."""
    
P = []
Q = []
M  = []
for i in range(1000): #Plotted for 2 sets
  N = 1000
  S0, I0, R0 = 999, 1, 0  # initial conditions: one infected, rest susceptible
  t = np.linspace(0, 99, 100) # Grid of time points (in days)
  y0 = S0, I0, R0 # Initial conditions vector
  # Integrate the SIR equations over the time grid, t.
  ret = odeint(deriv, y0, t, args=(N, beta[i], gamma[i]))
  S, I, R = ret.T
  P.append(S)
  Q.append(I)
  M.append(R)

  

In [ ]:
""" list to dataftrame """

S_value = pd.DataFrame(P)
I_value = pd.DataFrame(Q)
R_value = pd.DataFrame(M)

In [100]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
# define dataset
s_list = []
i_list = []
r_list = []
for i in range(1000):
  s_list.append(array(S_value.iloc[i][0:70]))
  i_list.append(array(I_value.iloc[i][0:70]))
  r_list.append(array(R_value.iloc[i][0:70]))


In [101]:
# reshape series
for i in range(1000):
  s_list[i] =s_list[i].reshape(70,1)
  i_list[i] =i_list[i].reshape(70,1)
  r_list[i] =r_list[i].reshape(70,1)

In [102]:
p = []
q = []
r = []
for i in range(1000):
  p.append(s_list[i])
  q.append(i_list[i])
  r.append(r_list[i])  



In [103]:
s = tuple(p)
t = tuple(q)
u = tuple(r)


In [104]:
# horizontally stack columns
dataset = hstack(s)
# define generator
n_input = 2
n_features = dataset.shape[1]
generator = TimeseriesGenerator(dataset, dataset, length=n_input, batch_size=1)
# number of samples
print('Samples: %d' % len(generator))

Samples: 68


In [105]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1000))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit_generator(generator1, steps_per_epoch=1, epochs=500, verbose=0)

In [106]:
list = []
P = [np.array(pd.DataFrame(dataset).iloc[68]).tolist(), np.array(pd.DataFrame(dataset).iloc[69]).tolist()]
P = np.array(P)
x_input = P.reshape((1, n_input, n_features))
list.append(model.predict(x_input, verbose=0))

In [107]:
Q = [np.array(pd.DataFrame(dataset).iloc[69]).tolist(),np.array(pd.DataFrame(list[0]).iloc[0]).tolist()]
Q = np.array(Q)
x_input = Q.reshape((1, n_input, n_features))
list.append(model.predict(x_input, verbose=0))


In [108]:
for i in range(28):
  Q = [np.array(pd.DataFrame(list[i]).iloc[0]).tolist(),np.array(pd.DataFrame(list[i+1]).iloc[0]).tolist()]
  Q = np.array(Q)
  x_input = Q.reshape((1, n_input, n_features))
  list.append(model.predict(x_input, verbose=0))


In [109]:
df = pd.DataFrame()
for i in range(30):
  df = pd.concat([df,pd.DataFrame(list[i])],axis = 0)


In [110]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,217.725159,214.249954,214.897888,221.189072,213.771484,218.081177,217.632675,934.896179,215.184402,220.107666,221.345718,217.733963,214.958603,224.322464,218.406357,220.751938,214.503540,951.519104,217.204422,219.684799,215.050385,220.474518,213.494644,215.138153,213.917892,220.841049,213.124100,215.664597,213.621811,210.483643,214.010040,216.632324,209.885208,218.414261,221.210098,216.448364,213.829071,231.529190,214.644348,216.605026,...,219.046036,217.675690,214.135956,1001.092651,212.340408,226.886368,214.596954,218.737885,216.225174,212.021057,220.793900,216.760757,214.335709,998.647827,214.074219,219.758972,217.966675,213.875305,213.597717,211.849457,983.935303,219.525116,1006.939270,218.763000,212.372910,222.366638,214.157333,218.485687,511.444214,214.904419,215.939880,219.935593,213.175262,218.347244,212.424469,219.079605,212.154358,707.435425,214.060913,226.676697
0,217.514908,215.895401,213.232697,217.559204,214.239792,215.892914,214.494064,930.557251,216.459457,221.046356,218.906265,214.803589,212.534637,222.802170,218.144440,218.561783,214.538040,948.603027,214.294510,222.548065,217.172546,221.119965,212.337265,211.330582,212.713364,221.656601,214.559143,214.289581,214.236465,211.992004,213.078156,214.472183,211.381210,218.299149,219.460617,217.569473,215.402359,227.411728,213.093674,214.807526,...,219.849213,220.309387,212.796738,999.439758,214.317337,226.372772,213.166168,218.213150,215.144989,212.957825,220.100632,216.508728,212.972397,996.282715,213.505600,216.528061,214.120850,215.357071,216.048889,212.507980,981.866638,220.041763,1006.085510,217.334274,213.173538,219.097809,212.797943,217.979599,508.468048,213.625336,213.797012,220.032639,214.547470,219.100876,212.875549,218.257736,213.648529,705.543518,214.736282,224.332520
0,217.293411,216.112717,212.474533,215.177292,213.626129,213.142334,213.727814,924.698059,216.010223,220.725006,218.256287,212.180817,209.617432,221.236191,216.842804,216.766403,214.930008,944.983887,212.664474,223.516800,218.532623,219.624390,210.764313,209.011002,211.572495,220.651413,215.039856,212.159012,212.476776,211.962997,210.998077,212.326096,211.403458,217.632568,217.846588,216.389404,216.383514,225.016861,211.754166,214.014847,...,219.241089,220.046997,212.353973,995.982544,214.809555,226.061996,212.372757,218.319397,214.154465,212.372757,217.748169,214.349197,211.965454,990.925293,212.418732,214.000519,212.620163,214.675293,216.539185,211.969757,977.733093,218.347763,1002.771484,216.015900,212.782623,216.930130,211.766464,216.329941,504.951996,213.097748,212.372879,220.315842,213.883698,219.619125,212.683151,218.017990,215.124542,703.890503,214.464417,222.782242
0,216.576645,215.412811,211.794907,214.523132,212.901855,212.357101,213.067108,921.330322,215.082642,219.707214,217.493607,211.475021,209.037445,220.428482,216.106430,216.004822,214.027679,941.620178,212.110352,222.586777,217.562119,218.635590,210.088882,208.482086,210.967743,219.739273,214.230392,211.440887,211.668381,211.367249,210.224869,211.619659,210.744781,216.881042,217.078110,215.431839,215.545334,224.402954,211.201477,213.514496,...,218.317337,219.072342,211.528015,992.308655,214.000580,225.301346,211.716858,217.580566,213.469330,211.474594,216.950256,213.431244,211.177368,987.366577,211.658524,213.343735,211.924988,213.725967,215.749039,211.230515,974.219421,217.548599,999.133484,215.278748,212.221680,216.121292,211.074249,215.330719,503.297180,212.476852,211.626083,219.621063,213.069794,218.967590,212.017456,217.323120,214.228287,701.347412,213.636856,222.011185
0,215.828812,214.656357,211.044632,213.792114,212.038757,211.621964,212.185287,917.991638,214.315933,218.877365,216.788132,210.646484,208.290466,219.594269,215.440033,215.223419

In [111]:
result = S_value.transpose() 

In [112]:
result[70:100]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
70,202.845993,202.8459,206.125009,202.8459,202.8459,202.8459,206.667216,950.347268,202.854122,202.846336,202.857517,202.8459,202.902110,220.437132,202.845901,202.8459,202.8459,957.094453,202.8459,225.665862,202.8459,202.8459,202.8459,203.260932,202.8459,202.8459,202.8459,202.845953,202.8459,202.8459,202.848943,206.458384,202.845930,202.8459,202.8459,202.8459,209.535598,232.814901,202.8459,202.8459,...,202.8459,202.8459,202.8459,980.662758,202.8459,229.619372,204.108897,202.8459,202.845904,213.239186,202.845961,202.845955,202.8459,982.582479,202.8459,202.8459,206.248635,202.8459,202.8459,202.8459,974.814913,202.8459,987.206596,202.8459,203.139853,202.8459,202.845913,202.8459,659.306396,202.845901,202.852338,202.8459,206.391019,202.845904,202.845974,206.992185,202.848414,818.532777,202.8459,229.171832
71,202.845971,202.8459,205.748978,202.8459,202.8459,202.8459,206.236255,948.089530,202.852585,202.846240,202.855391,202.8459,202.892870,218.782825,202.845900,202.8459,202.8459,955.209167,202.8459,223.591446,202.8459,202.8459,202.8459,203.202744,202.8459,202.8459,202.8459,202.845940,202.8459,202.8459,202.848339,206.048459,202.845922,202.8459,202.8459,202.8459,208.827464,230.188741,202.8459,202.8459,...,202.8459,202.8459,202.8459,979.991027,202.8459,227.236981,203.949129,202.8459,202.845903,212.195262,202.845946,202.845941,202.8459,981.999248,202.8459,202.8459,205.859988,202.8459,202.8459,202.8459,973.860838,202.8459,986.825425,202.8459,203.097402,202.8459,202.845910,202.8459,644.721926,202.845901,202.851115,202.8459,205.987911,202.845903,202.845956,206.528767,202.847909,809.166458,202.8459,226.823934
72,202.845954,202.8459,205.416151,202.8459,202.8459,202.8459,205.854005,945.740054,202.851335,202.846165,202.853655,202.8459,202.885150,217.285371,202.845900,202.8459,202.8459,953.249057,202.8459,221.707421,202.8459,202.8459,202.8459,203.152716,202.8459,202.8459,202.8459,202.845930,202.8459,202.8459,202.847855,205.685148,202.845917,202.8459,202.8459,202.8459,208.194570,227.795208,202.8459,202.8459,...,202.8459,202.8459,202.8459,979.297849,202.8459,225.068793,203.809587,202.8459,202.845902,211.256768,202.845935,202.845931,202.8459,981.398102,202.8459,202.8459,205.515819,202.8459,202.8459,202.8459,972.873516,202.8459,986.433955,202.8459,203.061082,202.8459,202.845907,202.8459,630.117016,202.845900,202.850125,202.8459,205.630735,202.845902,202.845943,206.117269,202.847506,799.507454,202.8459,224.687603
73,202.845941,202.8459,205.121546,202.8459,202.8459,202.8459,205.514937,943.295912,202.850318,202.846106,202.852236,202.8459,202.878698,215.929704,202.845900,202.8459,202.8459,951.211683,202.8459,219.996063,202.8459,202.8459,202.8459,203.109704,202.8459,202.8459,202.8459,202.845923,202.8459,202.8459,202.847467,205.363129,202.845912,202.8459,202.8459,202.8459,207.628867,225.613407,202.8459,202.8459,...,202.8459,202.8459,202.8459,978.582589,202.8459,223.095269,203.687706,202.8459,202.845901,210.412956,202.845926,202.845924,202.8459,980.778528,202.8459,202.8459,205.211019,202.8459,202.8459,202.8459,971.851898,202.8459,986.031920,202.8459,203.030008,202.8459,202.845905,202.8459,615.534692,202.845900,202.849323,202.8459,205.314235,202.845901,202.845932,205.751846,202.847184,789.564843,202.8459,222.743497
74,202.845931,202.8459,204.860759,202.8459,202.8459,202.8459,205.214156,940.754151,202.849492,202.846061,202.851076,202.8459,202.873307,214.702230,202.845900,202.8459,202.8459,949.094572,202.8459,218.441326,202.8459,202.8459,202.8459,203.072722,202.8459,202.8459,202.8459,202.845917,202.8459,202.8459,202.847156,205.077693,202.845909,202.8459,202.8459,202.8459,207.123179,223.624336,202.8459,202.8459,...,202.8459,202.8459,202.8459,977.844595,202.8459,221.298680,203.581249,202.845